In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/modified-dataset/modified_dataset.csv
/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv


In [4]:
pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModel, AutoTokenizer, GPT2Model,GPT2Tokenizer,GPT2LMHeadModel
from transformers import BertModel, BertTokenizer
import pandas as pd

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
device

device(type='cuda')

In [8]:
model_name_encoder = "bert-base-uncased"

In [9]:
encoder_tokenizer = BertTokenizer.from_pretrained(model_name_encoder)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
file_path = "/kaggle/input/metacognitive-dataset/metacognitive-dataset.csv"
df = pd.read_csv(file_path)

In [11]:
file_path = "/content/drive/MyDrive/metacognitive-dataset.csv"
df = pd.read_csv(file_path)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df.head(5)

,question,difficulty,prefer_solution,flaw_solution,random_col_1,random_col_2,random_col_3,random_col_4,random_col_5,random_col_6,...,random_col_9,random_col_10,random_col_11,random_col_12,random_col_13,random_col_14,random_col_15,random_col_16,metacognitive_vector,metacognitive_feedback
0,"=====Problem Statement=====\nGiven an integer,...",introductory,n = int(input().strip())\nw = len(str(bin(n))[...,w = len(str(bin(n))[2:])\nn = int(input().stri...,2,2,1,2,2,2,...,1,1,1,1,3,1,2,2,"[2, 2, 1, 2, 2, 2, 3, 2, 1, 1, 1, 1, 3, 1, 2, 2]","Your solution is very close to being correct, ..."
1,=====Problem Statement=====\nYou are given a s...,introductory,#!/usr/bin/env python3\n\ndef __starting_point...,#!/usr/bin/env python3\n\ndef __starting_point...,1,3,1,2,2,2,...,2,3,1,3,3,1,2,1,"[1, 3, 1, 2, 2, 2, 3, 2, 2, 3, 1, 3, 3, 1, 2, 1]",It's clear that you have a good understanding ...
2,=====Problem Statement=====\nYou are given a s...,introductory,A = set(input().split())\nn = int(input())\nc...,s = set(input().split())\nm = int(input())\nc...,1,1,2,3,3,3,...,1,3,1,1,1,2,1,1,"[1, 1, 2, 3, 3, 3, 2, 2, 1, 3, 1, 1, 1, 2, 1, 1]",Your approach to the problem shows a good unde...
3,=====Problem Statement=====\nYou are given a p...,introductory,"for i in range(1,int(input())): #More than 2 l...","for idx in range(1, int(input())):\n print(...",2,1,1,3,1,3,...,1,3,1,1,3,1,3,2,"[2, 1, 1, 3, 1, 3, 1, 2, 1, 3, 1, 1, 3, 1, 3, 2]",Your approach to the problem shows a good unde...
4,=====Function Descriptions=====\nsum\n\nThe su...,introductory,"import numpy\nn,m=list(map(int,input().split()...",Variable Renaming\n\n```\nimport numpy as np\n...,2,2,3,2,2,1,...,1,2,3,3,3,1,3,2,"[2, 2, 3, 2, 2, 1, 1, 3, 1, 2, 3, 3, 3, 1, 3, 2]",It appears you are on the right track with you...


In [12]:
# from torch.utils.data import DataLoader, Dataset

# class CustomDataset(Dataset):
#     def __init__(self, df, tokenizer):
#         self.df = df
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         row = self.df.iloc[idx]

#         description: row['description'],
#         student_code: row['student_code'],
#         feedback: row['feedback'],
#         metacognitive_feedback: row['metacognitive_feedback'],
#         metacognitive_vector: torch.tensor(
#             ast.literal_eval(row['metacognitive_profile']), dtype=torch.float
#         )
#         context = f"Description: {description} Student Code: {student_code} Feedback: {feedback}"

#         encoded_context = self.tokenizer(
#             context,
#             return_tensors='pt',
#             padding='max_length',
#             truncation=True,
#             max_length=512
#         )


#         return encoded_context , metacognitive_vector




In [ ]:
#dataset = CustomDataset(df, tokenizer)
dataloader = DataLoader(df, batch_size=1, shuffle=True)

In [ ]:
dataloader.head()

In [13]:
encoder = BertModel.from_pretrained(model_name_encoder)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [14]:
tokenizer_encoder = BertTokenizer.from_pretrained(model_name_encoder)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
class ContextEncoder(nn.Module):
    def __init__(self, bert_model_name='bert-base-uncased', output_dim=768):
        super(ContextEncoder, self).__init__()

        self.tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert_encoder = BertModel.from_pretrained(bert_model_name)

        self.fc = nn.Linear(self.bert_encoder.config.hidden_size, output_dim)

    def forward(self, description, student_code, feedback):

        context = f"Description: {description} Student Code: {student_code} Feedback: {feedback}"


        encoded_inputs = self.tokenizer(
            context,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=512
        )


        with torch.no_grad():
            context_hidden_states = self.bert_encoder(**encoded_inputs).last_hidden_state


        context_rep = context_hidden_states.mean(dim=1)
        context_rep = self.fc(context_rep)
        final_rep = context_rep.unsqueeze(1)

        return final_rep

In [16]:
context_encoder = ContextEncoder()

In [17]:
context_encoder

ContextEncoder(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [18]:
class PersonaEncoder(nn.Module):
    def __init__(self, metacognitive_dim=16, output_dim=768):
        super(PersonaEncoder, self).__init__()


        self.metacognitive_fc = nn.Linear(metacognitive_dim, output_dim)  # from 16 to 768


        self.final_fc = nn.Linear(output_dim, output_dim)

    def forward(self, metacognitive_vector):

        metacognitive_rep = self.metacognitive_fc(metacognitive_vector)


        final_rep = self.final_fc(metacognitive_rep)  # (batch_size, output_dim)
        persona_rep = final_rep.unsqueeze(1)

        return persona_rep

In [19]:
persona_encoder = PersonaEncoder().to(device)

In [20]:
persona_encoder

PersonaEncoder(
  (metacognitive_fc): Linear(in_features=16, out_features=768, bias=True)
  (final_fc): Linear(in_features=768, out_features=768, bias=True)
)

In [21]:
class PAALayer(nn.Module):
    def __init__(self, hidden_dimension = 768 , tau=0.5):
        super(PAALayer, self).__init__()
        self.hidden_dimenstion = hidden_dimension
        self.tau = tau

        # Linear layers for generating the initial weights (wpersona)
        self.fc = nn.Linear(2 * hidden_dimension, hidden_dimension)  # Concatenated hR and oP
        self.sigmoid = nn.Sigmoid()


    def forward(self, hR , oP, oC):
        #cross attention results
        Mp_input  = torch.cat([hR,oP], dim=-1)#1
        #print("Mp input:", Mp_input.shape)
        Mp = self.fc(Mp_input)#2
        #print("Mp:",Mp.shape)
        Wp = self.sigmoid(Mp) #3
        #print("Wp:",Wp.shape)

        #apply weighting = 4
        Mpersona = (Wp > self.tau).float()
        #print("Mpersona:",Mpersona.shape)
        Mcontext = (1 - Wp > self.tau).float()
        #print("Mcontext:",Mcontext.shape)

        #apply masking to the cross attention / element wise multiplication = 5
        oP_weighted = Mpersona * oP
        #print("oP_weighted:",oP_weighted.shape)
        oC_weighted = Mcontext * oC
        #print("oC_weighted:",oC_weighted.shape)

        #adding the cross attention results = 6
        HPAA = oP_weighted + oC_weighted #output resulted from PAA layers
        #print("HPAA:",HPAA.shape)
        return HPAA

In [22]:
PAA = PAALayer()

In [23]:
PAA

PAALayer(
  (fc): Linear(in_features=1536, out_features=768, bias=True)
  (sigmoid): Sigmoid()
)

In [24]:
class CustomTransformerBlock(nn.Module):
    def __init__(self, hidden_size, tau):
        super(CustomTransformerBlock, self).__init__()


        self.context_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)
        self.persona_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)

        self.paa_layer = PAALayer(hidden_dimension=hidden_size, tau=tau)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, 2048),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(2048, hidden_size),
            nn.LayerNorm(hidden_size)
        )



    def forward(self, hR, encoded_persona, encoded_context):

        # hR = hR.transpose(0, 1)
        # encoded_persona = encoded_persona.transpose(0, 1)
        # encoded_context = encoded_context.transpose(0, 1)

        # transformer_output = self.transformer_decoder_layer(hR, encoded_persona)


        oP, _ = self.persona_attn(hR, encoded_persona, encoded_persona)
        oC, _ = self.context_attn(hR, encoded_context, encoded_context)


        HPAA = self.paa_layer(hR, oP, oC)
        return HPAA


In [29]:
custom = CustomTransformerBlock(768,0.5)

In [30]:
custom

CustomTransformerBlock(
  (context_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (persona_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
  )
  (paa_layer): PAALayer(
    (fc): Linear(in_features=1536, out_features=768, bias=True)
    (sigmoid): Sigmoid()
  )
  (mlp): Sequential(
    (0): Linear(in_features=768, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=2048, out_features=768, bias=True)
    (4): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)

In [25]:
class PAAModel(nn.Module):
    def __init__(self, hidden_size=768, vocab_size = 50257 ,tau=0.5, max_length=512, num_transformer_blocks=6):
        super(PAAModel , self).__init__()
        self.hidden_size = hidden_size
        self.tau = tau
        self.vocab_size = vocab_size
        self.max_length = max_length
        self.num_transformer_blocks = num_transformer_blocks

        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        #self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.add_special_tokens({'pad_token': '[BOS]'})
        self.embedding = nn.Embedding(len(self.tokenizer), self.hidden_size)



        #self.transformer_decoder = nn.TransformerDecoderLayer(d_model = hidden_size , nhead=8)
        #self.decoder = nn.TransformerDecoder(self.transformer_decoder, num_layers=4)
        self.gpt2 = GPT2LMHeadModel.from_pretrained('gpt2')

        # for param in self.gpt2.parameters():
        #     param.requires_grad = False

        self.decoder = self.gpt2.transformer
        #self.decoder = GPT2LMHeadModel.from_pretrained('gpt2')



        # self.context_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)
        # self.persona_attn = nn.MultiheadAttention(hidden_size, num_heads=12, batch_first=True)





       # self.paa = PAALayer(hidden_dimension=hidden_size, tau=tau)

        self.transformer_blocks = nn.ModuleList([CustomTransformerBlock(hidden_size, tau) for _ in range(num_transformer_blocks)])
        self.final_fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, metacognitive_response , metacognitive_vector, encoded_context , encoded_persona):

        tokenized_response = self.tokenizer(
            metacognitive_response,
            return_tensors='pt',
            padding='max_length',
            truncation=True,
            max_length=self.max_length-16
        )['input_ids'].to(device)

        #print("tokenized response:" ,tokenized_response.shape)
        #print("tokenizer:" , self.tokenizer)

        metacognitive_response_emb = self.embedding(tokenized_response)
        #print("metacognitive response embedding shape:" , metacognitive_response_emb.shape)

        #print("metacognitive vector long:", metacognitive_vector.shape)
        # metacognitive_vector_ids = torch.tensor(metacognitive_vector)
        #print("metacognitive vector token ids shape:", tokenized_vector.shape)
        # metacognitive_vector_ids = metacognitive_vector_ids.expand(1, tokenized_response.shape[1])  # Adjust to the length of tokenized response
        metacognitive_vector_emb = self.embedding(metacognitive_vector)
        #print("metacognitive vector emb shape:" , metacognitive_vector_emb.shape)
        # print("metacognitive vector token ids shape:", metacognitive_vector_ids.shape)

        #deocder input - concatenate reposnse + vector = step 1
        decoder_input = torch.cat([metacognitive_response_emb, metacognitive_vector_emb], dim=1)
        #print("decoder input shape:" ,decoder_input.shape)

        #step 2 - self attention on input embedding
        # Use a dummy memory tensor filled with zeros if no encoder memory is provided
        #memory = torch.zeros((decoder_input.size(0), 1, self.hidden_size)).to(decoder_input.device)

        inputs_embeds = decoder_input

        transformer_output = inputs_embeds



        outputs = self.decoder(inputs_embeds=inputs_embeds, output_hidden_states=True)
        hR = (outputs.hidden_states)[-1]
        #hR = self.decoder(decoder_input , memory)
        #print("hR shape:" , hR.shape)

        transformer_output = hR
        for transformer_block in self.transformer_blocks:
            transformer_output = transformer_block(transformer_output, encoded_persona, encoded_context)

        #step 3 = cross attention
        # oP, _ = self.persona_attn(hR, encoded_persona, encoded_persona)
        # oC, _ = self.context_attn(hR, encoded_context, encoded_context)
        # #print("oP shape:" , oP.shape)
        # #print("oC shape:" , oC.shape)


        # #step 4 = apply PAA layer
        # HPAA = self.paa(hR , oP, oC)
        #print("HPAA shape:"  , HPAA.shape)

        #step 5 = linear output
        logits = self.final_fc(transformer_output)
        #print("logits shape:" , logits.shape)

        return logits

In [26]:
model = PAAModel()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [27]:
model.to(device)

PAAModel(
  (embedding): Embedding(50258, 768)
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2SdpaAttention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )

In [31]:
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [32]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [33]:
LOSS = torch.nn.CrossEntropyLoss()

In [47]:
num_epochs = 100

In [35]:
import ast

In [36]:
tokenizer.pad_token = tokenizer.eos_token

In [42]:
for batch_idx, (description, student_code, feedback, metacognitive_vector, target_labels) in enumerate(dataloader):
    print(description)

NameError: name 'dataloader' is not defined

In [33]:
df.columns

Index(['question', 'difficulty', 'prefer_solution', 'flaw_solution',
       'random_col_1', 'random_col_2', 'random_col_3', 'random_col_4',
       'random_col_5', 'random_col_6', 'random_col_7', 'random_col_8',
       'random_col_9', 'random_col_10', 'random_col_11', 'random_col_12',
       'random_col_13', 'random_col_14', 'random_col_15', 'random_col_16',
       'metacognitive_vector', 'metacognitive_feedback'],
      dtype='object')

In [37]:
df_train = df[:400]

In [45]:
print("hhh",end='\r')
print("fff",end='\r')

In [ ]:
for epoch in range(num_epochs):
    print("training started:", epoch)
    model.train()
    total_loss = 0

    for idx,row in df_train.iterrows():
        print(f"{idx+1} - {len(df_train)}", end='\r')
        description = row['question']
        student_code = row['flaw_solution']
        feedback = row['prefer_solution']
        metacognitive_feedback = row['metacognitive_feedback']
        metacognitive_vector= row['metacognitive_vector']
        #print("metacognitive_vector:" ,metacognitive_vector)

        metacognitive_tensor = torch.tensor([
            ast.literal_eval(metacognitive_vector) ], dtype=torch.float).to(device)

        #print("metacognitive vector shape:", metacognitive_tensor.shape)
        #print(metacognitive_tensor)
        #print("metacognitive vector shape unsqueeze:", metacognitive_tensor.unsqueeze(1).shape)
        target_labels = torch.tensor(
            tokenizer.encode(metacognitive_feedback, max_length=512, truncation=True, padding="max_length")
        ).to(device)

        #metacognitive_feedback = ast.literal_eval(metacognitive_feedback)
        #metacognitive_feedback_tensor = torch.tensor(metacognitive_feedback, dtype=torch.float32).unsqueeze(0)


        context_encoding = context_encoder(description, student_code , feedback).to(device)
        persona_encoding = persona_encoder(metacognitive_tensor).to(device)
        #print("encoded context:",context_encoding.shape)
        #print("encoded persona:",persona_encoding.shape)



        logits = model(feedback, metacognitive_tensor.long(), context_encoding, persona_encoding)
        #print("logits.shape" ,logits.shape)

        logits = logits.view(-1, logits.size(-1))
        target_ids = target_labels.view(-1)

        #print("target labels:" , (logits.shape, target_ids.shape))

        #print("Target label values:", target_labels)
        # print("Maximum target value:", target_labels.max())
        # print("Model vocab size:", model.vocab_size)
        # print("------------------------------------------------")
        # print("target:",target_ids)
        # print("------------------------------------------------")
        # print("logits:",logits)
        loss = LOSS(logits,target_ids)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"\nEpoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(df_train):.4f}")


training started: 0
400 - 400
Epoch [1/100], Loss: 4.4643
training started: 1
400 - 400
Epoch [2/100], Loss: 4.4641
training started: 2
400 - 400
Epoch [3/100], Loss: 4.4642
training started: 3
400 - 400
Epoch [4/100], Loss: 4.4643
training started: 4
400 - 400
Epoch [5/100], Loss: 4.4641
training started: 5
400 - 400
Epoch [6/100], Loss: 4.4644
training started: 6
400 - 400
Epoch [7/100], Loss: 4.4642
training started: 7
400 - 400
Epoch [8/100], Loss: 4.4643
training started: 8
400 - 400
Epoch [9/100], Loss: 4.4641
training started: 9
400 - 400
Epoch [10/100], Loss: 4.4643
training started: 10


In [ ]:
# Save model state dict
torch.save(model.state_dict(), 'paamodel.pth')

In [ ]:
# Initialize the model before loading
model = PAAModel()

# Load the saved state dict
model.load_state_dict(torch.load('paamodel.pth'))

# Set to evaluation mode if needed
#model.eval()

In [ ]:
def inference(model, context_encoder, persona_encoder, description, student_code, feedback, metacognitive_feedback,metacognitive_vector):
    model.eval()
    model.to(device) # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation for inference
        # Step 1: Convert the input data to the necessary tensors
        metacognitive_tensor = torch.tensor(
            [ast.literal_eval(metacognitive_vector)], dtype=torch.float
        ).to(device)

        # Step 2: Encode context and persona using the respective encoders
        context_encoding = context_encoder(description, student_code, feedback).to(device)  # Shape: [1, 768]
        persona_encoding = persona_encoder(metacognitive_tensor).to(device)  # Shape: [1, 768]

        # Step 3: Prepare the metacognitive response (this is the input for the decoder)
        # metacognitive_response = tokenizer.encode(
        #                         metacognitive_feedback,
        #                         padding=True,
        #                         truncation=True,
        #                         max_length=512,
        #                         return_tensors='pt'
        #                     )['input_ids']

        # Step 4: Pass everything through the model (no gradients needed for inference)
        logits = model(metacognitive_feedback, metacognitive_tensor.long(), context_encoding, persona_encoding)

        # Step 5: Convert logits to predictions (for simplicity, let's take the argmax here)
        predictions = torch.argmax(logits, dim=-1)  # Taking argmax along the vocabulary dimension

        # Step 6: If needed, convert the predicted tokens back to text (this step depends on your output format)
        predicted_tokens = predictions.squeeze().tolist()
        decoded_text = tokenizer.decode(predicted_tokens, skip_special_tokens=True)

        # Convert tokens to text (if using a tokenizer for language generation, replace this step)
        # For simplicity, assuming the output is a sequence of tokens
        return predicted_tokens,decoded_text





In [3]:
df.columns

NameError: name 'df' is not defined

In [ ]:
row = df.iloc[450]

description = row['question']
student_code = row['flaw_solution']
feedback = row['prefer_solution']
metacognitive_feedback = row['metacognitive_feedback']
metacognitive_vector= row['metacognitive_vector']

In [ ]:
predicted_tokens ,predictions= inference(model, context_encoder, persona_encoder, description, student_code, feedback, metacognitive_feedback,metacognitive_vector)

In [ ]:
predictions